# Basic Critique-Revise

You can also run this notebook online [on Noteable.io](https://app.noteable.io/published/41cf4f55-bd8c-4b7f-b3d4-547d5e601960/basic_critique_revise).

This is an basic example of correcting an LLM's output using a pattern called critique-revise, where we highlight what part of the output is wrong and re-query the LLM for a correction.

In the below example, we define a Zod schema to match an array of tasks we want the LLM to compose for a given input. Take note that we expect the output tasks to match one of a defined set of types:

In [1]:
Deno.env.set("OPENAI_API_KEY", "");
Deno.env.set("LANGCHAIN_API_KEY", "");
Deno.env.set("LANGCHAIN_TRACING_V2", "true");

import { z } from "npm:zod";

const zodSchema = z
  .object({
    tasks: z
      .array(
        z.object({
          title: z
            .string()
            .describe("The title of the tasks, reminders and alerts"),
          due_date: z
            .string()
            .describe("Due date. Must be a valid ISO date string with timezone"),
          task_type: z
            .enum([
              "Call",
              "Message",
              "Todo",
              "In-Person Meeting",
              "Email",
              "Mail",
              "Text",
              "Open House",
            ])
            .describe("The type of task"),
        })
      )
      .describe("The JSON for task, reminder or alert to create"),
  })
  .describe("JSON definition for creating tasks, reminders and alerts");


We convert this Zod schema to JSON schema and bind it to an OpenAI model as a function. We also set the `function_call` argument so that the function will always be called and the LLM will always attempt to match its output to the provided schema, regardless of input:

In [2]:
import { zodToJsonSchema } from "npm:zod-to-json-schema";

import { JsonOutputFunctionsParser } from "npm:langchain@0.0.173/output_parsers";
import { ChatOpenAI } from "npm:langchain@0.0.173/chat_models/openai";
import { PromptTemplate } from "npm:langchain@0.0.173/prompts";

const functionSchema = {
  name: "task-scheduler",
  description: "Schedules tasks",
  parameters: zodToJsonSchema(zodSchema)
};

const template = `Respond to the following user query to the best of your ability:

{query}`;

const generatePrompt = PromptTemplate.fromTemplate(template);

const taskFunctionCallModel = new ChatOpenAI({
  temperature: 0,
  modelName: "gpt-3.5-turbo",
}).bind({
  functions: [functionSchema],
  function_call: { name: "task-scheduler" },
});

const generateChain = generatePrompt
  .pipe(taskFunctionCallModel)
  .pipe(new JsonOutputFunctionsParser()) 
  .withConfig({ runName: "GenerateChain" });

Next, we invoke the chain on an input. We can see that despite the passed function schema, the model was influenced by the particular user input, and the the outputted task type does not match one of the defined types, and is instead `"reminder"`:

In [3]:
import { TraceGroup } from "npm:langchain@0.0.173/callbacks";

const traceGroup = new TraceGroup("CritiqueReviseChain");
const groupManager = await traceGroup.start();

const userQuery = `Set a reminder to renew our online property ads next week.`;

let result = await generateChain.invoke({
  query: userQuery,
}, { callbacks: groupManager });

console.log(result);

{
  tasks: [
    {
      title: "Renew online property ads",
      due_date: "next week",
      task_type: "Reminder"
    }
  ]
}


We use our original Zod schema's `safeParse` method as a convenient validator to show this. It includes an error object summarizing the problem:

In [4]:
const outputValidator = (output: unknown) => zodSchema.safeParse(output);

let validatorResult = outputValidator(result);

console.log(JSON.stringify(validatorResult, null, 2));

{
  "success": false,
  "error": {
    "issues": [
      {
        "received": "Reminder",
        "code": "invalid_enum_value",
        "options": [
          "Call",
          "Message",
          "Todo",
          "In-Person Meeting",
          "Email",
          "Mail",
          "Text",
          "Open House"
        ],
        "path": [
          "tasks",
          0,
          "task_type"
        ],
        "message": "Invalid enum value. Expected 'Call' | 'Message' | 'Todo' | 'In-Person Meeting' | 'Email' | 'Mail' | 'Text' | 'Open House', received 'Reminder'"
      }
    ],
    "name": "ZodError"
  }
}


Now, we define a revise chain that will attempt to fix the problem. We reuse the previously defined model with the bound function call arguments:

In [5]:
const reviseTemplate = `Original prompt:
--------------
{original_prompt}
--------------

Completion:
--------------
{completion}
--------------

Above, the completion did not satisfy the constraints given by the original prompt and provided schema.

Error:
--------------
{error}
--------------

Try again. Only respond with an answer that satisfies the constraints laid out in the original prompt and provided schema:`;

const revisePrompt = PromptTemplate.fromTemplate(reviseTemplate);

const reviseChain = revisePrompt
  .pipe(taskFunctionCallModel)
  .pipe(new JsonOutputFunctionsParser())
  .withConfig({ runName: "ReviseChain" });

And finally, we rerun the erroneous output with the original prompt, completion, and error message until it passes the validation successfully:

In [6]:
let fixCount = 0;

const formattedOriginalPrompt = await generatePrompt.format({
  query: userQuery
});

try {
  while (!validatorResult.success && fixCount < 5) {
    result = await reviseChain.invoke({
      original_prompt: formattedOriginalPrompt,
      completion: JSON.stringify(result),
      error: JSON.stringify(validatorResult.error),
    }, { callbacks: groupManager });
    validatorResult = outputValidator(result);
    fixCount++;
  }
} finally {
  await traceGroup.end();
}

console.log(JSON.stringify(validatorResult, null, 2));

{
  "success": true,
  "data": {
    "tasks": [
      {
        "title": "Renew online property ads",
        "due_date": "next week",
        "task_type": "Todo"
      }
    ]
  }
}


You can see an example LangSmith trace of the full chain here: https://smith.langchain.com/public/967974dd-ac6c-42a2-8796-da84589d06a5/r